# Recommender System

From scratch!

Datasets from: https://www.kaggle.com/datasets/gargmanas/movierecommenderdataset

In [44]:
import pandas as pd


In [45]:
df = pd.read_csv("ratings.csv")

In [46]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Traditional recommender systems:
- Collaborative filtering: Filters out data. Also uses other peoples data. Similar Items based on purchased item. Recommends other items that a similar user has bought


In [47]:
df['userId'].nunique()

610

In [48]:
df['userId'].max()

610

In [49]:
df['movieId'].max()

193609

In [50]:
df['movieId'].nunique()

9724

In [51]:
#Encode categorical values
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [52]:
df['userId_encoded'] = encoder.fit_transform(df['userId'])
df.head()

,userId,movieId,rating,timestamp,userId_encoded
0,1,1,4.0,964982703,0
1,1,3,4.0,964981247,0
2,1,6,4.0,964982224,0
3,1,47,5.0,964983815,0
4,1,50,5.0,964982931,0


In [53]:
df['movieId_encoded'] = encoder.fit_transform(df['movieId'])
df.head()

,userId,movieId,rating,timestamp,userId_encoded,movieId_encoded
0,1,1,4.0,964982703,0,0
1,1,3,4.0,964981247,0,2
2,1,6,4.0,964982224,0,5
3,1,47,5.0,964983815,0,43
4,1,50,5.0,964982931,0,46


In [54]:
#Build neural network
import tensorflow as tf

In [55]:
#Input layer
input_userId = tf.keras.layers.Input(shape=(1))
input_movieID = tf.keras.layers.Input(shape=(1))

#Embedding layer
embedding_userId = tf.keras.layers.Embedding(input_dim = df['userId_encoded'].nunique(), output_dim = 64)(input_userId)
embedding_movieId = tf.keras.layers.Embedding(input_dim = df['movieId_encoded'].nunique(), output_dim = 64)(input_movieID)

#Flatten layer
flatten_userId = tf.keras.layers.Flatten()(embedding_userId)
flatten_movieId = tf.keras.layers.Flatten()(embedding_movieId)

#Hidden layer
dense_user = tf.keras.layers.Dense(64, activation='relu')(flatten_userId)
dense_movie = tf.keras.layers.Dense(64, activation='relu')(flatten_movieId)

#Dot product
output = tf.keras.layers.Dot(axes=1)([dense_user, dense_movie])

output = tf.squeeze(output)

model = tf.keras.models.Model(inputs = [input_userId, input_movieID], outputs = output)

In [56]:
model.compile(optimizer = 'adam', loss='mse')

In [57]:
model.fit([df['userId_encoded'], df['movieId_encoded']], df['rating'], epochs=5, batch_size=64)

Epoch 1/5

1576/1576 [==============================] - 16s 9ms/step - loss: 1.3818
Epoch 2/5
1576/1576 [==============================] - 12s 8ms/step - loss: 0.7036
Epoch 3/5
1576/1576 [==============================] - 12s 8ms/step - loss: 0.6370
Epoch 4/5
1576/1576 [==============================] - 14s 9ms/step - loss: 0.5484
Epoch 5/5
1576/1576 [==============================] - 13s 8ms/step - loss: 0.4594
